In [2]:
# imports
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
from IPython.display import display

import py4DSTEM

In [20]:
# initialize arrays
ducky = np.load("data/clean-ducky-4D.npy")
amplitudes = np.sqrt(np.fft.ifftshift(ducky.reshape(-1,128,128),axes=(-2,-1)))

obj_array_init = np.ones((416,416),dtype=np.complex128)
probe_array_init = py4DSTEM.process.phase.utils.ComplexProbe(
    energy=80e3,
    gpts=(128,128),
    sampling=(0.125,0.125),
    semiangle_cutoff=20,
    defocus=500
).build(
)._array

probe_array_init *= np.sqrt(np.sum(ducky,axis=(-2,-1)).mean((0,1))/128/128)

overlap_init = np.fft.fft2(probe_array_init)[None]

In [16]:
x,y = np.meshgrid(
    np.arange(0,400,16) + 16,
    np.arange(0,400,16) + 16,
    indexing='ij'
)
positions = np.stack((x.ravel(),y.ravel()),axis=-1)
patch_row, patch_col = patch_indices(positions,(128,128),(416,416))

In [17]:
shuffled_order = np.arange(625)
np.random.shuffle(shuffled_order)

In [15]:
def patch_indices(positions, roi_shape, obj_shape):
    """Returns the vectorized row/col patch indices."""

    sx, sy = roi_shape
    x0 = np.round(positions[:, 0]).astype("int")
    y0 = np.round(positions[:, 1]).astype("int")

    x_ind = np.fft.fftfreq(sx, d=1 / sx).astype("int")
    y_ind = np.fft.fftfreq(sy, d=1 / sy).astype("int")

    row = (x0[:, None, None] + x_ind[None, :, None]) % obj_shape[0]
    col = (y0[:, None, None] + y_ind[None, None, :]) % obj_shape[1]

    return row, col